# Example: combining MOD file ion channels with rxd

A version of this notebook may be run online via Google Colab at https://tinyurl.com/neuron-rxd-and-mod
 (make a copy or open in playground mode).

## Overview

NEURON's reaction-diffusion infrastructure can be used to readily allow intracellular concentrations to respond to currents generated in MOD files, as long as:

- `nrn_region='i'` is specified for the `rxd.Region` (so that it knows it corresponds to the electrophysiology region of the inside of the cell), AND
the name and charge of the ion/etc are given in the `rxd.Species` declaration.
Satisfying the above two rules also allows MOD files to see intracellular concentrations.

- 3D extracellular concentrations also interoperate with electrophysiology automatically as long as name and charge are specified.

As a simple example, we consider a model with just a single point soma, of length and diameter 10 microns, with Hodgkin-Huxley kinetics (which uses the built in mod file `hh.mod`), and dynamic sodium (declared using rxd but without any additional kinetics).

## Setup NEURON library and imports

Let's import our usual NEURON libraries and definitions. Remember you can use either `um` or `µm` for micron.

In [ ]:
from neuron import n, rxd
from neuron.units import mV, ms, um, mM

## needed for standard run system
n.load_file("stdrun.hoc")

Now import `plotly`, a graphics library. (You could easily modify this code to use other graphics libraries like `matplotlib`, `plotnine`, or `bokeh`.)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Setup the model

In [ ]:
## define morphology
soma = n.Section(name="soma")
soma.L = soma.diam = 10 * um

## add ion channels (n.hh is built in, so always available)
n.hh.insert(soma)

## define cytosol. MUST specify nrn_region for concentrations to update
cyt = rxd.Region([soma], name="cyt", nrn_region="i")

## define sodium. MUST specify name and charge for concentrations to update
na = rxd.Species(cyt, name="na", charge=1)

Alternatively, we could have written `n.hh.insert(soma.wholetree())` to put Hodgkin-Huxley channels everywhere in the cell that the soma is part of, but since there is only one section, this is not required.

Let's also add an excitatory synapse to receive events (these will trigger the cell to spike).

In [ ]:
syn = n.ExpSyn(soma(0.5))
syn.tau = 1 * ms
syn.e = 0 * mV

## Add a stimulus

The spike events themselves (two events, 15 ms apart starting at h.t=10*ms):

In [ ]:
stim = n.NetStim()
stim.interval = 15 * ms
stim.number = 2
stim.start = 10 * ms

Send those events to our synapse:

In [ ]:
nc = n.NetCon(stim, syn)
nc.weight[0] = 0.001

## Setup recording variables

In [ ]:
t = n.Vector().record(n._ref_t)
v = n.Vector().record(soma(0.5)._ref_v)
na_vec = n.Vector().record(soma(0.5)._ref_nai)
ina = n.Vector().record(soma(0.5)._ref_ina)

## Initialize and run the simulation

In [ ]:
n.finitialize(-65 * mV)
n.continuerun(50 * ms)

## Plot it

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=t, y=v, name="v"), row=1, col=1)
fig.update_yaxes(title_text="v (mV)", row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=ina, name="ina"), row=2, col=1)
fig.update_yaxes(title_text="ina (mA/cm^2)", row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=na_vec, name="[Na+]"), row=3, col=1)
fig.update_xaxes(title_text="t (ms)", row=3, col=1)
fig.update_yaxes(title_text="[Na+] (mM)", row=3, col=1)

fig.show(renderer="notebook_connected")

## Note: On homeostatic mechanisms or the lack thereof

Without any additional homeostatic mechanisms (Hodgkin and Huxley did not model sodium concentration so they did not need to include homeostatic mechanisms for it), intracellular sodium concentration will not return to baseline, and each spike will move intracellular sodium concentration closer to the extracellular concentration. Potassium concentration in this model is constant as we did not declare a potassium rxd.Species, but if we did it would also approach its extracellular concentration with each spike, until eventually the cell is not able to fire action potentials anymore.

## Finally: The mod file

For any distributed mechanism, including the built-in `n.hh`, one can always get the source code for the corresponding mod file using the `.code` property; e.g.

In [ ]:
print(n.hh.code)